In [8]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import json
import glob
import os
import pandas as pd
import json

In [5]:
cd ../data/

/srv/home/christinedk/wp_internship/data


In [6]:
ls -lh editor_history

total 879G
-rw-r--r-- 1 christinedk wikidev 190G Mar  8 23:32 user_history-advert-meta-info.json
-rw-r--r-- 1 christinedk wikidev 161G Mar  8 22:50 user_history-autobiography-meta-info.json
-rw-r--r-- 1 christinedk wikidev 147G Mar  9 00:00 user_history-fanpov-meta-info.json
-rw-r--r-- 1 christinedk wikidev 193G Mar  8 22:19 user_history-peacock-meta-info.json
-rw-r--r-- 1 christinedk wikidev 191G Mar  8 21:40 user_history-weasel-meta-info.json


In [20]:
template = 'fanpov'
i = 0

with open('/srv/home/christinedk/wp_internship/data/editor_history/user_history-{}-meta-info.json'.format(template)) as f:
    editor = 'start'
    editor_lines = []
    
    for l in f:        
        json_obj_list = json.loads(l)
        if json_obj_list['event_user_id'] == editor:
            editor_lines.append(json_obj_list)
        else:
            with open('editor_split/{}.json'.format(editor), 'a+') as out_json_file:
                json.dump(editor_lines, out_json_file, indent=4)
            editor = json_obj_list['event_user_id']
            editor_lines = [json_obj_list]
            i += 1
            if i % 100 ==0:
                print(i)
                break
            

100


In [21]:
files = glob.glob('/srv/home/christinedk/wp_internship/data/editor_split/*.json')

In [22]:
sizes = [(f.split('/')[-1], os.stat(f).st_size/(1024*1024)) for f in files]
sizes = pd.DataFrame(sizes,columns=['user_id','size'])

In [23]:
sizes.sort_values(by='size',ascending=False)[:50]

,user_id,size
73,10728040.json,171.980059
11,10755432.json,32.496665
63,10727795.json,28.856299
77,10734192.json,23.387546
97,10743416.json,21.705446
90,10695509.json,19.125782
3,10747469.json,17.064280
14,10750750.json,16.738990
72,10722519.json,8.732622
36,10754397.json,4.907018


In [24]:
with open('editor_split/10750750.json','r') as f:
    editor_history = pd.DataFrame(json.load(f))

In [25]:
from math import log2
from datetime import datetime


def entropy(p):
    return -sum([p[i] * log2(p[i]) for i in range(len(p))])

In [28]:
editor_history['event_user_registration_timestamp'] = pd.to_datetime(editor_history.event_user_registration_timestamp)
editor_history = editor_history.sort_values(by='event_timestamp')
TEST_TIME = pd.to_datetime(datetime.now()) # Note: Make this tag add time and remove history when tag added????? Multiple per user???

# Membership of user groups
ngroups = editor_history['num_groups'].values[-1]

# # articles edited
narticles = editor_history.page_id.nunique()

# total # revisions
nedits = len(editor_history)

In [33]:
# max contribution frac & entropy
edits_per_article = editor_history.groupby('page_id').size()
cf = edits_per_article/sum(edits_per_article)
max_cf = cf.max()
entropy_cf = entropy(cf.values)

# # past blocks
past_blocks = editor_history.num_blocks_historical.values[-1]

# # current blocks
curr_blocks = editor_history.num_curr_blocks.values[-1]

# # edits reverted by others
edits_reverted_own = editor_history.revision_is_identity_reverted.sum()

# # revisions editor reverted
edits_reverted_others = editor_history.revision_is_identity_revert.sum()

# Time since registration
time_since_registration = TEST_TIME - editor_history.event_user_registration_timestamp.values[-1] 

# Ratio of page edits to Talk page posts
posts_per_type = editor_history.groupby('page_namespace').size()
talk_article_ratio = posts_per_type.loc[0.0]/posts_per_type.loc[1.0]

# # conflicting interactions

In [35]:
# editor-editor